## Deadlocks (Bloqueo Mutuo)

**Escenario:** Sesión A bloquea recurso X y espera recurso Y; Sesión B bloquea Y y espera X → impasse.

SQL Server detecta deadlocks automáticamente y aborta una transacción (víctima).

### Estrategias de prevención:
1. **Orden consistente de acceso**: siempre actualiza tablas en el mismo orden
2. **Transacciones cortas**: menos tiempo con locks = menos riesgo
3. **Índices apropiados**: reduce filas bloqueadas
4. **SNAPSHOT isolation**: evita locks de lectura

```sql
-- Ejemplo de patrón propenso a deadlock (NO hacer)
-- Sesión A:
BEGIN TRAN; UPDATE tabla1 WHERE id=1; UPDATE tabla2 WHERE id=2; COMMIT;
-- Sesión B (simultánea):
BEGIN TRAN; UPDATE tabla2 WHERE id=2; UPDATE tabla1 WHERE id=1; COMMIT;
-- Deadlock! Cada sesión tiene un recurso que la otra necesita

-- Solución: orden consistente
-- Ambas sesiones: UPDATE tabla1 primero, luego tabla2
```

## 🟢 Ejercicio Básico
Escribe una transacción que:
1. Descuente 10 unidades de `fact_inventario` para producto_id = 105
2. Inserte una venta en `fact_ventas` con esas 10 unidades
3. Haga ROLLBACK si stock insuficiente

## 🟠 Ejercicio Intermedio
Explica qué isolation level usarías para:
- Dashboard de métricas que se actualiza cada minuto (no crítico si datos tienen 1s de delay)
- Proceso de facturación que debe ser exacto al centavo
- Reporte de auditoría que tarda 5 minutos (no puede bloquear transacciones operativas)

## 🔴 Ejercicio Avanzado
Diagnostica y resuelve un deadlock:
1. Habilita trace flag 1222 para capturar deadlock graphs
2. Simula un deadlock con 2 sesiones
3. Analiza el XML del deadlock
4. Propón solución (reordenar operaciones, cambiar isolation level, índices)

---

## Errores Comunes

❌ **No usar transacciones**: cambios parciales en operaciones multi-paso
❌ **Transacciones largas**: mantienen locks mucho tiempo, bloquean otros usuarios
❌ **READ UNCOMMITTED en datos críticos**: lees valores que luego desaparecen
❌ **No manejar deadlocks en código**: aplicación crashea en vez de reintentar
❌ **Olvidar COMMIT**: transacción queda abierta, locks infinitos

**Siguiente:** `06_planes_ejecucion.ipynb` → leer y optimizar query plans

In [ ]:
-- Ejemplo de READ UNCOMMITTED (lectura sucia permitida)
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT producto_id, cantidad FROM dbo.fact_inventario WHERE producto_id = 101;
-- Lee datos incluso si otra sesión los está modificando sin commit
-- Ventaja: no espera locks, más rápido
-- Riesgo: puede leer valor que luego hace rollback

-- Volver a default
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;

-- Ejemplo de SNAPSHOT (usa versionado de filas)
-- Requiere habilitar en la BD: ALTER DATABASE MiDB SET ALLOW_SNAPSHOT_ISOLATION ON;
-- SET TRANSACTION ISOLATION LEVEL SNAPSHOT;
-- BEGIN TRANSACTION;
-- SELECT * FROM fact_ventas WHERE fecha = '2024-01-01';
-- -- Otra sesión modifica estas ventas
-- SELECT * FROM fact_ventas WHERE fecha = '2024-01-01';
-- -- Obtiene los MISMOS datos (snapshot al inicio de la transacción)
-- COMMIT;

/*
💡 SNAPSHOT es ideal para reportes largos:
✅ No bloquea escrituras (otras sesiones pueden modificar datos)
✅ Consistencia de lectura (siempre ves el snapshot inicial)
❌ Costo: SQL Server mantiene versiones de filas en tempdb
*/

## Niveles de Aislamiento (Isolation Levels)

Controlan qué tan "aislada" está una transacción de otras concurrentes.

| Nivel | Permite Lectura Sucia | Lectura No Repetible | Fantasmas | Uso Típico |
|-------|----------------------|---------------------|-----------|-----------|
| **READ UNCOMMITTED** | ✅ Sí (lee cambios no confirmados) | ✅ Sí | ✅ Sí | Reportes aproximados, no críticos |
| **READ COMMITTED** (default) | ❌ No | ✅ Sí | ✅ Sí | Mayoría de aplicaciones OLTP |
| **REPEATABLE READ** | ❌ No | ❌ No | ✅ Sí | Cálculos que requieren consistencia |
| **SERIALIZABLE** | ❌ No | ❌ No | ❌ No | Transacciones críticas (lento) |
| **SNAPSHOT** | ❌ No | ❌ No | ❌ No | Alternativa sin bloqueos (usa versionado) |

### Problemas de concurrencia:
- **Lectura sucia (dirty read)**: lees datos que luego hacen rollback
- **Lectura no repetible**: lees la misma fila 2 veces y obtiene valores distintos
- **Fantasmas (phantoms)**: ejecutas misma query 2 veces y obtiene distinto número de filas

In [ ]:
-- Ejemplo de transacción: transferencia de inventario entre productos
BEGIN TRANSACTION;

-- Descontar stock del producto A
UPDATE dbo.fact_inventario
SET cantidad = cantidad - 50
WHERE producto_id = 101 AND cantidad >= 50;

-- Verificar que se actualizó (@@ROWCOUNT = filas afectadas)
IF @@ROWCOUNT = 0
BEGIN
    ROLLBACK TRANSACTION;
    RAISERROR('Stock insuficiente en producto 101', 16, 1);
    RETURN;
END

-- Agregar stock al producto B
UPDATE dbo.fact_inventario
SET cantidad = cantidad + 50
WHERE producto_id = 102;

-- Confirmar cambios
COMMIT TRANSACTION;
PRINT 'Transferencia exitosa';

/*
💡 Sin BEGIN TRAN / COMMIT:
- Si falla UPDATE del producto B, el descuento de A ya se aplicó → inconsistencia
- Con transacción: si algo falla, ROLLBACK revierte TODO (atomicidad)

⚠️ Dentro de una transacción, SQL Server mantiene locks en las filas modificadas
hasta COMMIT o ROLLBACK (esto bloquea otras sesiones que quieran modificarlas)
*/

# 3.5 Transacciones y Bloqueos - ACID, Isolation Levels, Deadlocks

## 🎯 ¿Para qué?
En sistemas multiusuario, múltiples sesiones modifican datos simultáneamente. Las transacciones garantizan **consistencia** (datos no corruptos) y **aislamiento** (sesiones no se pisan). Los bloqueos (locks) son el mecanismo que SQL Server usa para coordinar acceso concurrente.

## 📚 ¿Por qué importa?

Sin transacciones:
- Usuario A actualiza precio de producto mientras usuario B lee el mismo registro → B lee valor intermedio inconsistente
- Transferencia bancaria: se descuenta de cuenta A pero falla al acreditar en cuenta B → dinero desaparece

Con transacciones:
- **Atomicidad**: todo o nada (rollback si falla una parte)
- **Consistencia**: datos respetan reglas (constraints, FK)
- **Aislamiento**: sesiones concurrentes no se interfieren
- **Durabilidad**: cambios confirmados sobreviven a fallas del servidor

## 🔧 Propiedades ACID